In [11]:
import random
import pickle
import numpy as np
import pandas as pd
from debugpy.common.timestamp import current
from nltk.tokenize import RegexpTokenizer
from numpy import dtype
from numpy.random import choice

from optree.functools import partial
from pygments.lexers.asn1 import word_sequences
from  tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop


In [4]:
text_df = pd.read_csv("fake_or_real_news.csv")

In [6]:
text_df

,id,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [7]:
text = list(text_df.text.values)
joined_text = " ".join(text)

In [81]:
partial_text = joined_text[:1000000]

In [12]:
tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(partial_text.lower())

In [13]:
tokens

['daniel',
 'greenfield',
 'a',
 'shillman',
 'journalism',
 'fellow',
 'at',
 'the',
 'freedom',
 'center',
 'is',
 'a',
 'new',
 'york',
 'writer',
 'focusing',
 'on',
 'radical',
 'islam',
 'in',
 'the',
 'final',
 'stretch',
 'of',
 'the',
 'election',
 'hillary',
 'rodham',
 'clinton',
 'has',
 'gone',
 'to',
 'war',
 'with',
 'the',
 'fbi',
 'the',
 'word',
 'unprecedented',
 'has',
 'been',
 'thrown',
 'around',
 'so',
 'often',
 'this',
 'election',
 'that',
 'it',
 'ought',
 'to',
 'be',
 'retired',
 'but',
 'it',
 's',
 'still',
 'unprecedented',
 'for',
 'the',
 'nominee',
 'of',
 'a',
 'major',
 'political',
 'party',
 'to',
 'go',
 'war',
 'with',
 'the',
 'fbi',
 'but',
 'that',
 's',
 'exactly',
 'what',
 'hillary',
 'and',
 'her',
 'people',
 'have',
 'done',
 'coma',
 'patients',
 'just',
 'waking',
 'up',
 'now',
 'and',
 'watching',
 'an',
 'hour',
 'of',
 'cnn',
 'from',
 'their',
 'hospital',
 'beds',
 'would',
 'assume',
 'that',
 'fbi',
 'director',
 'james',
 'c

In [14]:
unique_tokens = np.unique(tokens)
unique_tokens_index = {tokens:idx for idx, tokens in enumerate(unique_tokens)}

In [15]:
unique_tokens_index

{np.str_('2016'): 0,
 np.str_('2020'): 1,
 np.str_('5'): 2,
 np.str_('60'): 3,
 np.str_('a'): 4,
 np.str_('abc'): 5,
 np.str_('abcpolitics'): 6,
 np.str_('abedin'): 7,
 np.str_('about'): 8,
 np.str_('aboutface'): 9,
 np.str_('abuses'): 10,
 np.str_('accused'): 11,
 np.str_('accusing'): 12,
 np.str_('act'): 13,
 np.str_('ad'): 14,
 np.str_('admits'): 15,
 np.str_('ads'): 16,
 np.str_('afraid'): 17,
 np.str_('after'): 18,
 np.str_('afternoon'): 19,
 np.str_('against'): 20,
 np.str_('age'): 21,
 np.str_('agency'): 22,
 np.str_('agents'): 23,
 np.str_('ago'): 24,
 np.str_('ahead'): 25,
 np.str_('alive'): 26,
 np.str_('all'): 27,
 np.str_('allegations'): 28,
 np.str_('allies'): 29,
 np.str_('allowed'): 30,
 np.str_('already'): 31,
 np.str_('also'): 32,
 np.str_('amendment'): 33,
 np.str_('americans'): 34,
 np.str_('an'): 35,
 np.str_('and'): 36,
 np.str_('announced'): 37,
 np.str_('anthony'): 38,
 np.str_('any'): 39,
 np.str_('anywhere'): 40,
 np.str_('apolitical'): 41,
 np.str_('appearance

In [30]:
n_words = 10
input_words =[]
next_words = []

for i in range(len(tokens)- n_words):
    input_words.append(tokens[i:i+n_words])
    next_words.append(tokens[i+n_words])


In [19]:
input_words

[['daniel',
  'greenfield',
  'a',
  'shillman',
  'journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center'],
 ['greenfield',
  'a',
  'shillman',
  'journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is'],
 ['a',
  'shillman',
  'journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is',
  'a'],
 ['shillman',
  'journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is',
  'a',
  'new'],
 ['journalism',
  'fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is',
  'a',
  'new',
  'york'],
 ['fellow',
  'at',
  'the',
  'freedom',
  'center',
  'is',
  'a',
  'new',
  'york',
  'writer'],
 ['at',
  'the',
  'freedom',
  'center',
  'is',
  'a',
  'new',
  'york',
  'writer',
  'focusing'],
 ['the',
  'freedom',
  'center',
  'is',
  'a',
  'new',
  'york',
  'writer',
  'focusing',
  'on'],
 ['freedom',
  'center',
  'is',
  'a',
  'new',
  'york',
  'writer',
  'focusing',
  'on',
  'radical'],
 ['center',
  'is',
  'a',
  

In [32]:
x=np.zeros((len(input_words), n_words,len(unique_tokens)), dtype=bool)
y=np.zeros((len(next_words),len(unique_tokens)), dtype=bool)

In [22]:
x

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, Fal

In [33]:
y

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [34]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        x[i, j, unique_tokens_index[word]] = 1
    y[i, unique_tokens_index[next_words[i]]] = 1

In [35]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words,len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation('softmax'))

C:\Users\hamra\PycharmProjects\Text Generation AI\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [37]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01), metrics=['accuracy'])
model.fit(x, y, batch_size=128, epochs=30, shuffle=True)

Epoch 1/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.0801 - loss: 5.2106
Epoch 2/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.1077 - loss: 4.7430
Epoch 3/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.0979 - loss: 4.4504
Epoch 4/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.1550 - loss: 4.0850
Epoch 5/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.1648 - loss: 3.8680
Epoch 6/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.2030 - loss: 3.5181
Epoch 7/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.2937 - loss: 2.9785
Epoch 8/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3661 - loss: 2.7068
Epoch 9/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.4601 - loss: 2.3101
Epoch 10/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5484 - loss: 1.9876
Epoch 11/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6771 - loss: 1.6599
Epoch 12/30
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy:

In [38]:
model.save('model.h5')

In [39]:
model = load_model('model.h5')

In [55]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    x = np.zeros((1, n_words,len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        x[0, i, unique_tokens_index[word]] = 1
    prediction = model.predict(x)[0]
    return np.argpartition(prediction, n_best)[-n_best:]


In [66]:
possible = predict_next_word("he will have to look into thing and he", 5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


In [78]:
print({unique_tokens[idx] for idx in possible})

{np.str_('2020'), np.str_('you'), np.str_('zero'), np.str_('2016'), np.str_('your')}


In [79]:
def generate_text(input_text,text_length, creativity=3):
    word_sequences = input_text.split()
    current=0
    for _ in range(text_length):
        sub_sequences = "".join(tokenizer.tokenize("".join(word_sequences).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choices(predict_next_word(sub_sequences, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequences.append(choice)
        current==1
    return " ".join(word_sequences)


In [82]:
generate_text("she is working at",100, 5)

'she is working at elect then allegations bigger freedom very wake assaults not don wreck meant co attacking destroy speakerryan greenfield beds which figurehead targeting tak article film fighting interesting emails short presidential hatch never good be whatever anywhere weiner rodham where wreck countless clearly allegations development lit director idea emailing here 2016 t senator tak boston has ahead anywhere covert he supporting truths illegality crime react so lambasting awkward appearance senator print emailing criminal staggering then colleagues being through 60 principles comey it agency desperation presidential come investigators didn hospital personal of table reid emails hard co cunning step principles which re nobody'